In [1]:
import os
from pathlib import Path
import wget
from tqdm.auto import tqdm
import numpy as np
import torch
from torch.cuda.amp import GradScaler, autocast
from torch.nn import CrossEntropyLoss, Conv2d, BatchNorm2d
from torch.optim import SGD, lr_scheduler
import torchvision
import warnings
import argparse
warnings.filterwarnings('ignore')

/home/jinxulin/anaconda3/envs/miss/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Resnet9
class Mul(torch.nn.Module):
    def __init__(self, weight):
        super(Mul, self).__init__()
        self.weight = weight
    def forward(self, x): return x * self.weight


class Flatten(torch.nn.Module):
    def forward(self, x): return x.view(x.size(0), -1)


class Residual(torch.nn.Module):
    def __init__(self, module):
        super(Residual, self).__init__()
        self.module = module
    def forward(self, x): return x + self.module(x)


def construct_rn9(num_classes=10):
    def conv_bn(channels_in, channels_out, kernel_size=3, stride=1, padding=1, groups=1):
        return torch.nn.Sequential(
                torch.nn.Conv2d(channels_in, channels_out, kernel_size=kernel_size,
                            stride=stride, padding=padding, groups=groups, bias=False),
                torch.nn.BatchNorm2d(channels_out),
                torch.nn.ReLU(inplace=True)
        )
    model = torch.nn.Sequential(
        conv_bn(3, 64, kernel_size=3, stride=1, padding=1),
        conv_bn(64, 128, kernel_size=5, stride=2, padding=2),
        Residual(torch.nn.Sequential(conv_bn(128, 128), conv_bn(128, 128))),
        conv_bn(128, 256, kernel_size=3, stride=1, padding=1),
        torch.nn.MaxPool2d(2),
        Residual(torch.nn.Sequential(conv_bn(256, 256), conv_bn(256, 256))),
        conv_bn(256, 128, kernel_size=3, stride=1, padding=0),
        torch.nn.AdaptiveMaxPool2d((1, 1)),
        Flatten(),
        torch.nn.Linear(128, num_classes, bias=False),
        Mul(0.2)
    )
    return model

In [3]:

def get_dataloader(batch_size=256, num_workers=8, split='train', shuffle=False, augment=True):
    if augment:
        transforms = torchvision.transforms.Compose(
                        [torchvision.transforms.RandomHorizontalFlip(),
                         torchvision.transforms.RandomAffine(0),
                         torchvision.transforms.ToTensor(),
                         torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                          (0.2023, 0.1994, 0.201))])
    else:
        transforms = torchvision.transforms.Compose([
                         torchvision.transforms.ToTensor(),
                         torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                          (0.2023, 0.1994, 0.201))])
        
    is_train = (split == 'train')
    dataset = torchvision.datasets.CIFAR10(root='/tmp/cifar/',
                                           download=True,
                                           train=is_train,
                                           transform=transforms)

    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         shuffle=shuffle,
                                         batch_size=batch_size,
                                         num_workers=num_workers)
    
    return loader

In [4]:
def train(model, loader, lr=0.4, epochs=24, momentum=0.9,
          weight_decay=5e-4, lr_peak_epoch=5, label_smoothing=0.0, model_id=0):
    
    opt = SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    iters_per_epoch = len(loader)
    # Cyclic LR with single triangle
    lr_schedule = np.interp(np.arange((epochs+1) * iters_per_epoch),
                            [0, lr_peak_epoch * iters_per_epoch, epochs * iters_per_epoch],
                            [0, 1, 0])
    scheduler = lr_scheduler.LambdaLR(opt, lr_schedule.__getitem__)
    scaler = GradScaler()
    loss_fn = CrossEntropyLoss(label_smoothing=label_smoothing)

    for ep in range(epochs):
        print(f"Epoch {ep} of {epochs}")
        for it, (ims, labs) in enumerate(loader):
            ims = ims.cuda()
            labs = labs.cuda()
            opt.zero_grad(set_to_none=True)
            with autocast():
                out = model(ims)
                loss = loss_fn(out, labs)

            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
            scheduler.step()
        if ep in [12, 15, 18, 21, 23]:
            torch.save(model.state_dict(), f'./checkpoints/sd_{model_id}_epoch_{ep}.pt')
        
    return model

In [5]:

os.makedirs('./checkpoints', exist_ok=True)
loader_for_training = get_dataloader(batch_size=512, split='train', shuffle=True)

# you can modify the for loop below to train more models
for i in tqdm(range(1), desc='Training models..'):
    model = construct_rn9().to(memory_format=torch.channels_last).cuda()
    model = train(model, loader_for_training, model_id=i)

Files already downloaded and verified


Training models..:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0 of 24
Epoch 1 of 24
Epoch 2 of 24
Epoch 3 of 24
Epoch 4 of 24
Epoch 5 of 24
Epoch 6 of 24
Epoch 7 of 24
Epoch 8 of 24
Epoch 9 of 24
Epoch 10 of 24
Epoch 11 of 24
Epoch 12 of 24
Epoch 13 of 24
Epoch 14 of 24
Epoch 15 of 24
Epoch 16 of 24
Epoch 17 of 24
Epoch 18 of 24
Epoch 19 of 24
Epoch 20 of 24
Epoch 21 of 24
Epoch 22 of 24
Epoch 23 of 24


Training models..: 100%|██████████| 1/1 [00:48<00:00, 48.65s/it]


In [6]:
print(len(loader_for_training.dataset))

50000


# Make a list of checkpoints

In [12]:

ckpt_files = sorted(list(Path('./checkpoints').rglob('*.pt')))
ckpts = [torch.load(ckpt, map_location='cpu') for ckpt in ckpt_files]

In [13]:

model = construct_rn9().to(memory_format=torch.channels_last).cuda()
model.load_state_dict(ckpts[-1])
model = model.eval()

In [14]:
loader = get_dataloader(split='val', augment=False)
print(len(loader.dataset))
model.eval()

with torch.no_grad():
    total_correct, total_num = 0., 0.
    for ims, labs in tqdm(loader):
        ims = ims.cuda()
        labs = labs.cuda()
        with autocast():
            out = model(ims)
            total_correct += out.argmax(1).eq(labs).sum().cpu().item()
            total_num += ims.shape[0]

    print(f'Accuracy: {total_correct / total_num * 100:.1f}%')

Files already downloaded and verified
10000


100%|██████████| 40/40 [00:00<00:00, 62.25it/s]

Accuracy: 91.3%


In [15]:
batch_size = 128
loader_train = get_dataloader(batch_size=batch_size, split='train')

Files already downloaded and verified


In [17]:
from trak import TRAKer
print(len(loader_train.dataset))
traker = TRAKer(model=model,
                task='image_classification',
                proj_dim=4096,
                train_set_size=len(loader_train.dataset))

ERROR:TRAK:Could not use CudaProjector.
Reason: /home/jinxulin/anaconda3/envs/miss/lib/python3.8/site-packages/fast_jl.cpython-38-x86_64-linux-gnu.so: undefined symbol: _ZNK3c106SymInt13toSymNodeImplEv
ERROR:TRAK:Defaulting to BasicProjector.
INFO:STORE:Existing model IDs in /home/jinxulin/MISS/Classification/trak_results: [0, 1, 2, 3, 4]
INFO:STORE:Model IDs that have been finalized: [0, 1, 2, 3, 4]
INFO:STORE:Existing TRAK scores:
INFO:STORE:quickstart: /home/jinxulin/MISS/Classification/trak_results/scores/quickstart.mmap


50000


# Compute TRAK features for train data


In [19]:
print(len(ckpts))

1


In [20]:

for model_id, ckpt in enumerate(tqdm(ckpts)):
    print(model_id)
    traker.load_checkpoint(ckpt, model_id=model_id)
    for batch in tqdm(loader_train):
        batch = [x.cuda() for x in batch]
        traker.featurize(batch=batch, num_samples=batch[0].shape[0])

traker.finalize_features()

  0%|          | 0/1 [00:00<?, ?it/s]

0


Finalizing features for all model IDs..: 100%|██████████| 5/5 [00:00<00:00, 2991.66it/s]


In [21]:
loader_targets = get_dataloader(batch_size=batch_size, split='val', augment=False)
print(len(loader_targets.dataset))
for model_id, ckpt in enumerate(tqdm(ckpts)):
    traker.start_scoring_checkpoint(exp_name='quickstart',
                                    checkpoint=ckpt,
                                    model_id=model_id,
                                    num_targets=len(loader_targets.dataset))
    for batch in loader_targets:
        batch = [x.cuda() for x in batch]
        traker.score(batch=batch, num_samples=batch[0].shape[0])

scores = traker.finalize_scores(exp_name='quickstart')
print(scores.shape)

Files already downloaded and verified
10000


Finalizing scores for all model IDs..: 100%|██████████| 5/5 [00:03<00:00,  1.48it/s]
INFO:STORE:Saving scores in /home/jinxulin/MISS/Classification/trak_results/scores/quickstart.mmap


(50000, 10000)


In [22]:

from scipy.stats import spearmanr

In [23]:
def eval_correlations(scores, tmp_path):    
    # masks_url = 'https://www.dropbox.com/s/x76uyen8ffkjfke/mask.npy?dl=1'
    # margins_url = 'https://www.dropbox.com/s/q1dxoxw78ct7c27/val_margins.npy?dl=1'

    masks_path = Path(tmp_path).joinpath('mask.npy')
    # wget.download(masks_url, out=str(masks_path), bar=None)
    # num masks, num train samples
    masks = torch.as_tensor(np.load(masks_path, mmap_mode='r')).float()
    
    margins_path = Path(tmp_path).joinpath('val_margins.npy')
    # wget.download(margins_url, out=str(margins_path), bar=None)
    # num , num val samples
    margins = torch.as_tensor(np.load(margins_path, mmap_mode='r'))

    val_inds = np.arange(10_000)
    preds = masks @ scores
    rs = []
    ps = []
    for ind, j in tqdm(enumerate(val_inds)):
        r, p = spearmanr(preds[:, ind], margins[:, j])
        rs.append(r)
        ps.append(p)
    rs, ps = np.array(rs), np.array(ps)
    print(f'Correlation: {rs.mean():.3f} (avg p value {ps.mean():.6f})')
    return rs.mean()

In [26]:
# corr = eval_correlations(scores, '.')
tmp_path = './tmp'
masks_path = Path(tmp_path).joinpath('mask.npy')
masks = torch.as_tensor(np.load(masks_path, mmap_mode='r')).float()
print(masks.shape)

margins_path = Path(tmp_path).joinpath('val_margins.npy')
margins = torch.as_tensor(np.load(margins_path, mmap_mode='r'))
print(margins.shape)

torch.Size([6011, 50000])
torch.Size([6011, 10000])
